# keyboard_config

> Review-specific keyboard building blocks for assembly into a KeyboardManager

In [ ]:
#| default_exp components.keyboard_config

In [ ]:
#| export
from typing import Tuple

# Keyboard navigation library
from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager

# Card stack library
from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.keyboard.actions import (
    create_card_stack_focus_zone, create_card_stack_nav_actions,
)

## Keyboard Parts Builder

Returns `(zone, actions, modes)` tuple for assembly into a `KeyboardManager`.
Review uses simple navigation only (no sub-modes like segmentation's split mode).

In [ ]:
#| export
def create_review_kb_parts(
    ids:CardStackHtmlIds,  # Card stack HTML IDs
    button_ids:CardStackButtonIds,  # Card stack button IDs for navigation
    config:CardStackConfig,  # Card stack configuration
) -> Tuple[FocusZone, tuple, tuple]:  # (zone, actions, modes)
    """Create review-specific keyboard building blocks."""
    # Card stack zone from library
    # onReviewFocusChange callback handles audio playback on navigation
    card_zone = create_card_stack_focus_zone(
        ids=ids,
        on_focus_change="onReviewFocusChange",
    )

    # Card stack navigation actions from library (arrows, page jump, first/last, width)
    nav_actions = create_card_stack_nav_actions(
        zone_id=card_zone.id,
        button_ids=button_ids,
        config=config,
    )
    
    # Replay action — Space key to replay current segment's audio
    replay_action = KeyAction(
        key=" ",  # Space
        js_callback="replayReviewSegment",
        zone_ids=(card_zone.id,),
        description="Replay segment",
        hint_group="Audio",
    )

    # Combine all actions
    actions = nav_actions + (replay_action,)
    modes = ()  # No sub-modes

    return card_zone, actions, modes

## Zone Manager Factory

Creates a complete `ZoneManager` for the review step (single-zone, no zone switching).

In [ ]:
#| export
def create_review_keyboard_manager(
    ids:CardStackHtmlIds,  # Card stack HTML IDs
    button_ids:CardStackButtonIds,  # Card stack button IDs for navigation
    config:CardStackConfig,  # Card stack configuration
) -> ZoneManager:  # Configured keyboard zone manager
    """Create the keyboard zone manager for the review step."""
    # Get review-specific building blocks
    review_zone, review_actions, review_modes = create_review_kb_parts(
        ids=ids,
        button_ids=button_ids,
        config=config,
    )
    
    # Assemble into ZoneManager (single zone, no zone switching)
    return ZoneManager(
        zones=(review_zone,),
        actions=review_actions,
        modes=review_modes,
        initial_zone_id=review_zone.id,
        state_hidden_inputs=True,
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()